In [1]:
import pandas as pd
WaypointNode = pd.read_csv("WaypointNode.csv", usecols=["ID", "Name_lang", "SafeLocID"])
WaypointSafeLocs = pd.read_csv("WaypointSafeLocs.csv")
WaypointEdge = pd.read_csv("WaypointEdge.csv", usecols=["Start", "End", "PlayerConditionID"])

In [2]:
WaypointNode_Loc = (pd.merge(WaypointNode, WaypointSafeLocs,
                           how="inner",
                           left_on="SafeLocID",
                           right_on="ID",
                           suffixes=["", "_WSL"])
                        .drop(columns=["ID_WSL"]))

In [3]:
waypoint_graph = (pd.merge(WaypointEdge, WaypointNode_Loc,
                          how="inner",
                          left_on="Start",
                          right_on="ID",
                          suffixes=["", "_start"])
                     .drop(columns=["SafeLocID", "ID"])
                     .merge(WaypointNode_Loc,
                           how="inner",
                           left_on="End",
                           right_on="ID",
                           suffixes=["", "_end"])
                     .drop(columns=["SafeLocID", "ID"]))

In [4]:
import numpy as np
def distance3d(x1, y1, z1, x2, y2, z2):
    return np.sqrt(np.power(x2 - x1, 2) +
                np.power(y2 - y1, 2) +
                np.power(z2 - z1, 2))

In [5]:
walk_graphs = []
for MapID in WaypointNode_Loc.MapID.unique():
    WaypointsInMap = WaypointNode_Loc[WaypointNode_Loc.MapID == MapID]
    xjoin = pd.merge(WaypointsInMap, WaypointsInMap, how="cross", suffixes=["_start", "_end"])
    xjoin["Distance"] = distance3d(xjoin['Pos[0]_start'], xjoin['Pos[1]_start'], xjoin['Pos[2]_start'],
                                   xjoin['Pos[0]_end'], xjoin['Pos[1]_end'], xjoin['Pos[2]_end'])
    xjoin = xjoin.drop(columns=["SafeLocID_end", "SafeLocID_start"])
    xjoin = (xjoin.rename(columns={
                    "ID_start": "Start",
                    "ID_end": "End"
                })
             .rename(columns=lambda colname: colname.replace("_start", "")))
    walk_graphs.append(xjoin)

In [6]:
graph = pd.concat(walk_graphs + [waypoint_graph], ignore_index=True)
graph["EdgeName"] = graph["Start"].map(str) + " " + graph["End"].map(str)

In [7]:
PlayerCondition = pd.read_csv("PlayerCondition.csv", usecols=["ID", "RaceMask"])
ChrRaces = pd.read_csv("ChrRaces.csv", usecols=["ID", "PlayableRaceBit"])
ChrRaces = ChrRaces[ChrRaces["PlayableRaceBit"] != -1] # remove unplayable races